### Adding a single trip to the database
Create a method `add_trip` that accepts the following parameters:

* `driver` : trip driver as a string in `Last name`, `Given name` format
* `pickup_datetime` : datetime of pickup as string with format `"hh:mm:ss,DD-MM-YYYY"`
* `dropoff_datetime` : datetime of dropoff as string with format `"hh:mm:ss,DD-MM-YYYY"`
* `passenger_count` : number of passengers as integer
* `pickup_loc_name` : zone as a string, (e.g., Pine View, Legazpi Village)
* `dropoff_loc_name` : zone as a string, (e.g., Pine View, Legazpi Village)
* `trip_distance` : distance in meters (float)
* `fare_amount` : amount paid by passenger (float)

The method should append the trip data to the end of `trips.csv` , if it exists, or creates it, otherwise.

The `trip_id` is last `trip_id` in the file + 1, or `1` if there's no trip in the file yet.

The `driver_id` is the corresponding `driver_id` in `drivers.csv` based on the case-insensitive matches of `given_name` and `last_name` . It should append the driver in
drivers.csv if the driver is not yet there. The `driver_id` of a new driver is _last driver_id in the file_ + 1, or `1` if there's no driver in the file yet. The method should
return the `trip_id` or raise a `SakayDBError` exception if the trip is already in `trips.csv`. A trip is said to be in `trips.csv` if there is a trip that matches the `driver` (case-insensitive), `pickup_datetime`, `dropoff_datetime`, `passenger_count`, `pickup_loc_id`, `dropoff_loc_id`, `trip_distance` and `fare_amount`.

In [1]:
import numpy as np
import pandas as pd

import os.path
from pathlib import Path


In [2]:
def check_create_file(filename, columns):
    if not os.path.exists(filename):
        with open(filename, mode='w', encoding='utf-8') as f:
            f.write(','.join(columns) + '\n')
    return


In [3]:
def get_driver_id(driver):
    last_name, given_name = [x.strip() for x in driver.split(',')]
    
    fn = 'drivers_test.csv'
    df_drivers = pd.read_csv(fn)
    cond = ((df_drivers['given_name'].str.lower() == given_name.lower()) &
            (df_drivers['last_name'].str.lower() == last_name.lower()))

    if cond.any():
        driver_id = df_drivers[cond]['driver_id'].item()
        return driver_id
    else:
        return None


In [4]:
def get_loc_id(location):
    fn = 'locations.csv'
    df_locations = pd.read_csv(fn)
    cond = (df_locations['loc_name'].str.lower() == location.lower())
    
    if cond.any():
        loc_id = df_locations[cond]['location_id'].item()
        return loc_id
    else:
        return None


In [5]:
def get_trip_id(driver, pickup_datetime, dropoff_datetime, passenger_count,
                pickup_loc_name, dropoff_loc_name, trip_distance, fare_amount):
    fn = 'trips_test.csv'
    df_trips = pd.read_csv(fn)
    cond = ((df_trips['driver_id'] == get_driver_id(driver)) &
            (df_trips['pickup_datetime'] == pickup_datetime) &
            (df_trips['dropoff_datetime'] == dropoff_datetime) &
            (df_trips['passenger_count'] == passenger_count) &
            (df_trips['pickup_loc_id'] == get_loc_id(pickup_loc_name)) &
            (df_trips['dropoff_loc_id'] == get_loc_id(dropoff_loc_name)) &
            (df_trips['trip_distance'] == trip_distance) &
            (df_trips['fare_amount'] == fare_amount))
    
    if cond.any():
        trip_id = df_trips[cond]['trip_id'].item()
        return trip_id
    else:
        return None


In [6]:
def add_driver(driver):
    # get driver lastname and given name
    last_name, given_name = [x.strip() for x in driver.split(',')]

    fn = 'drivers_test.csv'
    cols = ['driver_id', 'given_name', 'last_name']

    # check if 'divers.csv' exists; 
    check_create_file(fn, cols)

    # load drivers df
    df_drivers = pd.read_csv(fn)

    # set driver_id
    if pd.isnull(df_drivers['driver_id'].max()):
        driver_id = 1
    else:
        driver_id = df_drivers['driver_id'].max() + 1

    # log driver details
    if get_driver_id(driver) is None:
        data = {
            'driver_id' : [driver_id],
            'given_name': [given_name],
            'last_name' : [last_name]
        }
        pd.DataFrame(data).to_csv(fn, mode='a', index=False, header=False)
        print('driver added')
        return
    else:
        print('driver already in db')
        return # raise SakayDBError(ValueError)


In [7]:
def add_trip(driver, pickup_datetime, dropoff_datetime, passenger_count,
             pickup_loc_name, dropoff_loc_name, trip_distance, fare_amount):
    fn = 'trips_test.csv'
    cols = ['trip_id', 'driver_id', 'pickup_datetime',
            'dropoff_datetime', 'passenger_count', 'pickup_loc_id',
            'dropoff_loc_id', 'trip_distance', 'fare_amount']
    
    # check if 'trips.csv' exists; 
    check_create_file(fn, cols)
    
    # load df_trips
    df_trips = pd.read_csv(fn)
    
    # add driver to drivers.csv
    add_driver(driver)

    # set trip_id
    if pd.isnull(df_trips['trip_id'].max()):
        trip_id = 1
    else:
        trip_id = df_trips['trip_id'].max() + 1
    
    # verify and log trip
    if get_trip_id(driver, pickup_datetime, dropoff_datetime, passenger_count,
                   pickup_loc_name, dropoff_loc_name, trip_distance, fare_amount) is None:
        trip_data = {
            'trip_id': [trip_id],
            'driver_id' : [get_driver_id(driver)],
            'pickup_datetime' : pickup_datetime,
            'dropoff_datetime' : dropoff_datetime,
            'passenger_count' : [passenger_count],
            'pickup_loc_id' : [get_loc_id(pickup_loc_name)],
            'dropoff_loc_id' : [get_loc_id(dropoff_loc_name)],
            'trip_distance' : [trip_distance],
            'fare_amount' : [fare_amount]
        }
                
        pd.DataFrame(trip_data).to_csv(fn, encoding='utf-8', mode='a', index=False, header=False)
        print('trip added')
        return
    else:
        print('trip already in db')
        return # raise SakayDBError(ValueError)


In [ ]:
# driver = 'Lazaro, BJ'
# pickup_datetime = '09:00:00,03-09-2022'
# dropoff_datetime = '09:45:00,03-09-2022'
# passenger_count = 3
# pickup_loc_name = 'Melvin'
# dropoff_loc_name = 'Loyola Heights'
# trip_distance = 12.1
# fare_amount = 321

# add_trip(driver, pickup_datetime, dropoff_datetime, passenger_count,
#          pickup_loc_name, dropoff_loc_name, trip_distance, fare_amount)